In [11]:
# parse dev data function
def ParseJson(data_json):
    paragraphs = data_json['data'][0]['paragraphs']
    idx_list = list()
    context_list = list()
    ques_list = list()
    ans_list = list()
    for para in paragraphs:
        context = para['context']
        idx = para['qas'][0]['id']
        ques = para['qas'][0]['question']
        ans = '|'.join([ans['text'] for ans in para['qas'][0]['answers']])
        idx_list.append(idx)
        context_list.append(context)
        ques_list.append(ques)
        ans_list.append(ans)
    return idx_list, context_list, ques_list, ans_list

In [9]:
# dev data
import json
dev_file = '/home/t-honli/data/squad_EQnA/dev-v1.1.json' 
dev_data = json.load(open(dev_file, "r"))

In [13]:
import pandas as pd
idx, context, ques, ans = ParseJson(dev_data)
dev_pd = pd.DataFrame({'id':idx, 'context':context, 'question':ques, 'ground_truth':ans}, columns=['id', 'context', 'question', 'ground_truth'])
dev_pd.head(5)

,id,context,question,ground_truth
0,9200ae8372eadcb43e5cfb816f185ef9,Start Up & Rev of a 2006 Acura RL SH - AWD.Thi...,2006 acura rl horsepower,290 HP
1,b7d7518b27e56604c9b868ba25565222,The income tax withholding rate remains at 4.2...,2015 michigan withholding tax rate,4.25 %
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon c...,5rr hair color,Red
3,92484e0e63e92f1412854e9516b8f8c3,Effects depend on the substance : hydrogen per...,acid that burns through skin,nitric acid|hydrogen peroxide
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who pl...",actor who plays jamie lannister,Nikolaj Coster|Waldau


In [14]:
# bidaf ans
import json
import os
file = '/home/t-honli/bi-att-flow/out/basic/00_EQnA/answer/test-020000.json'
bidaf = json.load(open(file, "r"))

In [16]:
dev_pd['bidaf'] = dev_pd.apply(lambda row: bidaf[row['id']], axis=1)

In [19]:
# prod model
import pandas as pd
col_names = ['id', 'Query', 'Url', 'Answer', 'AnswerTokenList', 'ParaseSpan', 'phrase', 'Label', 'Probability']
prod = pd.read_csv('/home/t-honli/data/squad_EQnA/EQnA_Highlighting_Test_ProdModel.tsv', header=None, sep='\t', names=col_names, dtype=str).fillna('')

print (len(prod))

# 根据answer，query，url生成hash_id
import hashlib
def GetHashCode(context):
    hash = hashlib.md5()
    hash.update(context.encode('utf-8'))
    return hash.hexdigest()
prod['hash_id'] = prod.apply(lambda row: GetHashCode(row['Answer'] + ' ' + row['Query'] + ' ' + row['Url']), axis=1)

prod = prod[prod.apply(lambda row: len(row['phrase'])<32, axis=1)]
print (len(prod))
prod.head(5)

15560
15454


,id,Query,Url,Answer,AnswerTokenList,ParaseSpan,phrase,Label,Probability,hash_id
0,00021900a221a0b2e5c9a923c4f13813,where does the water from mississippi's rivers...,http://en.wikipedia.org/wiki/Mississippi_River,"In the United States, the Mississippi River dr...","[""In"",""the"",""United"",""States"","","",""the"",""Missi...",79:79,rivers,0,0.01860236,9b6bef3c4ce179302bbad6ddf9b98832
1,0004dd703e44213707c923e548627774,good foods for pregnant women,http://www.netdoctor.co.uk/health_advice/facts...,"During pregnancy, a woman's body needs more ir...","[""During"",""pregnancy"","","",""a"",""woman"",""'s"",""bo...",4:4,woman,0,0.1246661,ac6273ec9adf9778fb33f09501f43874
2,00052a47dccf1667995ebffb3019b565,what continent is belize on,http://www.belize.com/where-is-belize,Belize is located on the Caribbean coast of no...,"[""Belize"",""is"",""located"",""on"",""the"",""Caribbean...",19:19,88°45,0,0.02859148,2110154eb61df6e89f85c33c204492e0
3,00099640760fac2092756da3714c60a9,when was tom seaver born,http://www.baseball-almanac.com/players/player...,"Tom Seaver was born on Friday, November 17, 19...","[""Tom"",""Seaver"",""was"",""born"",""on"",""Friday"","",""...",17:17,Seaver,0,0.001268676,a272a8fdd6b0e63d75e39f732639c31f
4,000e8ade636d16933ddc53486a59a976,disease caused by deer ticks,http://www.niaid.nih.gov/topics/tickborne/Page...,"Ticks transmit ehrlichiosis and anaplasmosis, ...","[""Ticks"",""transmit"",""ehrlichiosis"",""and"",""anap...",0:0,Ticks,0,0.008481221,54ee3a3b1b57174ee8ee50625d20ee74


In [23]:
# 获取hash_id, [(phrase,score)]
multi_phrase_dict = {}
def getPhrase(row):
    if row['hash_id'] not in multi_phrase_dict:
        multi_phrase_dict[row['hash_id']] = list()
    multi_phrase_dict[row['hash_id']].append(row['phrase'] + ":" + row['Probability'])
prod.apply(getPhrase, axis=1)
print (len(prod))

15454


In [43]:
# sort
for idx in list(multi_phrase_dict.keys()):
    multi_phrase_dict[idx].sort(key=lambda x: float(x.split(':')[-1]), reverse=True)

In [44]:
print (list(multi_phrase_dict.items())[:5])

[('9b6bef3c4ce179302bbad6ddf9b98832', ['Alabama:0.8559828', 'Rio Grande:0.7438805', 'Saint Lawrence River:0.7311112', 'Tombigbee:0.6697654', 'Chattahoochee:0.5843887', 'the Atlantic Ocean:0.4905324', 'Red River of the North:0.4180078', 'the United States:0.3122597', 'the Great Lakes:0.305788', 'waterways:0.2183529', 'the Mississippi:0.07368357', 'drained:0.04473107', 'coastal:0.02576688', 'rivers:0.02443846', 'rivers:0.01860236', 'regions:0.01731697']), ('ac6273ec9adf9778fb33f09501f43874', ['spinach:0.9554551', 'broccoli:0.94595', 'green vegetables:0.9092412', 'wholemeal bread:0.8100643', 'blood:0.6137482', 'produce:0.5851527', 'iron:0.4314162', 'placenta:0.2960912', 'iron:0.2656902', 'nutrition:0.2498231', 'pregnancy:0.1734539', 'woman:0.1246661', 'body:0.06024067']), ('2110154eb61df6e89f85c33c204492e0', ['South America:0.9047099', 'North America:0.6378803', 'Yucatán Peninsula:0.1944177', 'the Caribbean:0.1486141', 'Prime Meridian:0.1198179', 'the equator:0.0431998', '17°15:0.02910043

In [45]:
dev_pd['prod'] = dev_pd.apply(lambda row: '|'.join(multi_phrase_dict[row['id']]), axis=1)

In [47]:
pd.set_option('max_colwidth', 200)
dev_pd.head(5)

,id,context,question,ground_truth,bidaf,prod
0,9200ae8372eadcb43e5cfb816f185ef9,"Start Up & Rev of a 2006 Acura RL SH - AWD.This has a 3.5L V6 making a decent 290 HP while paired to a 5 Speed Auto.The interior in this was very classy , roomy , & Comfortable .",2006 acura rl horsepower,290 HP,290 HP,290 HP:0.8148856|V6:0.4650716|AWD.This:0.3202039|3.5L:0.1977224|5:0.1862928|has a:0.02482028|2006:0.01019631|Acura RL:0.009642786
1,b7d7518b27e56604c9b868ba25565222,"The income tax withholding rate remains at 4.25 % for tax year 2015 . However , the personal exemption amount for tax year 2015 will change to $4,000 . You may continue to use 2014 Michigan Income...",2015 michigan withholding tax rate,4.25 %,4.25 %,"4.25 %:0.858085|$4,000:0.6614105|2014:0.03836213|personal exemption:0.0170537|Withholding:0.008434641|withholding:0.007704477|Income Tax:0.006366469|income tax:0.00618484|2015:0.005863357|2015:0.0..."
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon color expertise with this Vidal Sassoon Pro Series Hair Color in 5RR Medium Vibrant Red . It offers 100 percent gray coverage in a permanent color form...,5rr hair color,Red,gray,Red:0.9268106|gray:0.9099237|Vidal Sassoon:0.0242435|100 percent:0.01703781|Vidal Sassoon:0.01560702|color:0.006009114|color:0.004669665|Hair Color:0.004049229|5RR:0.003362776
3,92484e0e63e92f1412854e9516b8f8c3,"Effects depend on the substance : hydrogen peroxide removes a bleached layer of skin , while nitric acid causes a characteristic color change to yellow in the skin . Chemical burns may occur throu...",acid that burns through skin,nitric acid|hydrogen peroxide,hydrogen peroxide,hydrogen peroxide:0.4229763|nitric acid:0.2648291|yellow:0.06255874|Chemical burns:0.0581477|color:0.02692936|body:0.01474041|may:0.007937214|skin:0.00594707|skin:0.005661041|skin:0.005624547
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who plays Jaime Lannister in HBO 's popular television series Game Of Thrones , has been spotted in Hobart , where he has spent time taking in the city 's c...",actor who plays jamie lannister,Nikolaj Coster|Waldau,Nikolaj Coster,Jaime:0.7372439|Nikolaj Coster:0.7275336|Waldau:0.05069283|Hobart:0.02077893|HBO:0.01916428|Game Of Thrones:0.008346976|television series:0.005669255|Actor:0.003533427|has been:0.003437618


In [48]:
dev_pd.to_csv('./sample.tsv', header=True, index=None, sep='\t')